In [1]:
import sqlite3
import pandas as pd
from src import paths,sql_querys,converting, pipelines, square_sum, split
import joblib
from sklearn.metrics import accuracy_score, recall_score

In [2]:
conn = sqlite3.connect(paths.db_path)
cursor = conn.cursor()
query = sql_querys.query_training_model
df = pd.read_sql_query(query, conn)
df.max_balance = pd.to_numeric(df.max_balance)
df.age=pd.to_numeric(df.age)
df = converting.converter(df)

In [4]:

age_pipeline = joblib.load(paths.age_pipeline_path)
missing_age_indices = df['age'].isnull()
predicted_ages = age_pipeline.predict(df[missing_age_indices])

df.loc[missing_age_indices, 'age'] = predicted_ages

In [5]:
query1 = (sql_querys.query_square_sum_train)
df = square_sum.square_sum(df,query1,conn)


In [ ]:
target_column = 'poutcome'
X_train, X_test, y_train, y_test = split.split(df, target_column)


num_features = ['age', 'balance_diff_square_sum', 'max_balance']
labeled_features = ['has_deposits', 'loan', 'has_mortgage','education' ]
RFC_params = {'random_state':42, 'n_estimators': 150, 'min_samples_split': 10,
                                            'min_samples_leaf':2, 'max_features': 'sqrt', 'max_depth': 10, 'criterion':'entropy', 'bootstrap': False}

pipeline = pipelines.main_pipeline(num_features,labeled_features,RFC_params)

In [9]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100), "Recall: {:.2f}%".format(recall * 100))

Accuracy: 72.17% Recall: 76.26%
